# Dataset Preparation

On this notebook we will:
- Split the videos into its frames and crop them using the midbody position
- Extract the labels from the DeepLabCutFiles

In [1]:
import os
import pandas as pd
import numpy as np
import cv2
from glob import glob

In [2]:
# Set and create the source and destination folders
videos_src_folder = '../data/raw/NewVideos'
csvs_src_folder = '../data/raw/NewCSVS'
dataset_dest_folder = '../data/NewProcessed/ImageDatasetRGB'

try:
    if not os.path.exists(os.path.dirname(dataset_dest_folder)):
        os.mkdir(dataset_dest_folder)
        os.mkdir(os.path.join(dataset_dest_folder, 'features'))
        os.mkdir(os.path.join(dataset_dest_folder, 'labels'))    

except OSError as err:
    print(err)
    print('Dataset Destination Folders already exists')

In [12]:
# Read the content from the source folders
csvs = sorted(os.listdir(csvs_src_folder))
videos = sorted(os.listdir(videos_src_folder))[1:]

In [14]:
if '.DS_Store' or 'DS_Store' in csvs:
    csvs = np.delete(csvs, 0)

In [15]:
csvs

array(['1.csv', '2.csv', '3.csv', '4.csv', '5.csv', 'Animal61980.csv',
       'Animal62418.csv'], dtype='<U15')

In [16]:
videos

['1.mp4',
 '2.mp4',
 '3.mp4',
 '4.mp4',
 '5.mp4',
 'Video_Animal61980_10min.mp4',
 'Video_Animal62418_10min.mp4']

We define a method to automatically find the central position of the mouse and crop the image.

In [17]:
def crop_image(image, expansion):
    
    # Start by converting the image to Gray scale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Rescale the values to highlight constrasted areas
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    
    # Crop the box
    thresh2 = np.invert(thresh)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    close = cv2.morphologyEx(thresh2, cv2.MORPH_CLOSE, kernel, iterations=2)
    
    cnts = cv2.findContours(close, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    
    # We want to avoid noise, for that we will just consider the biggest countour since it is going to be the box
    c = 0
    c_len = len(cnts[0])
    for i in range(len(cnts[1:])):
        if len(cnts[i]) > c_len:
            c = i
            c_len = len(cnts[i])          
    
    # Obtain bounding rectangle to get the box coordinates
    x,y,w,h = cv2.boundingRect(cnts[c])

    # If the box is way to big, we won't need to crop it, else we will 
    # If the box isnot detected by the filter, then we can consider it is big enough to not crop it
    if not (x+w - x) < image.shape[0]//4:
        image = image[y:y+h, x:x+w]
        thresh = thresh[y:y+h, x:x+w]

    # We can now look for the mouse contour
    cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
   
    # We want to avoid noise, for that we will just consider the biggest countour since it is going to be the mouse, as long as it is not
    # to close to the extremes of the image, since that would mean it is an external object
    c = 0
    c_len = len(cnts[0])
    for i in range(len(cnts[1:])):
        if len(cnts[i]) > c_len:
            # Obtain bounding rectangle to get measurements
            x,y,w,h = cv2.boundingRect(cnts[i])
            if not (y < 100) or not (y+h > image.shape[0]-100):
                c = i
                c_len = len(cnts[i])

    # Crate a bounding box arround the mouse
    x,y,w,h = cv2.boundingRect(cnts[c])
    
    # Find centroid, this will be the center position of the mouse
    M = cv2.moments(cnts[c])            
    cX = int(M["m10"] / (M["m00"] + 1e-10))
    cY = int(M["m01"] / (M["m00"]+ 1e-10))
    midbody = [cX,cY]
    top = max(0, midbody[0] - expansion) - max(0, midbody[0] + expansion - image.shape[0])
    bottom = min(image.shape[0], midbody[0] + expansion) + max(0, expansion - midbody[0])
    left = max(0, midbody[1] - expansion) - max(0, midbody[1] + expansion - image.shape[1])
    right = min(image.shape[1], midbody[1] + expansion) + max(0, expansion - midbody[1])
   
    return image[left:right,top:bottom]

Then, we can start processing the videos. 

In [20]:
expansion = 80
behaviours = ['Grooming', 'Rearing']
fps = 10

for csv, video in zip(csvs, videos):

    print(video)
    
    # Read columns from csvs corresponding to the behaviours
    df = pd.read_csv(os.path.join(csvs_src_folder, csv), header=0, usecols = behaviours)
    # Change voids per 0 
    df.fillna(0, inplace=True)
    # Change data format
    df = df.astype(int)
    # Reset row indexes
    df.reset_index(inplace=True, drop=True)
    # Save df as csv
    df.to_csv(os.path.join(dataset_dest_folder, 'labels', csv.split('.')[0] + '.csv'), index=False)
    print(csv)
    print('DF: ', df.shape)

    # Get video frames
    os.mkdir(os.path.join(dataset_dest_folder, 'features', csv.split('.')[0]))
    vidcap = cv2.VideoCapture(os.path.join(videos_src_folder, video))              

    # Get the video FPS rate
    fps_in = vidcap.get(cv2.CAP_PROP_FPS)
    print('Video FPS: ', fps_in)

    # Start processing each frame
    success,image = vidcap.read()

    frames_in = 0
    frames_out = 0
    count = 0
    count_df = 0
    drop_indx = []
    
    while success:
        #If the video already has the standard FPS, we don't have to do anything 
        if  fps_in == fps:
            # Build frame name
            frame_name = 'frame'
            for i in range(4-len(str(count))):
                frame_name += '0'
            frame_name += str(count) + '.jpg'
    
            # Crop image so mouse is postioned in the center
            frame = crop_image(image, expansion)
            
            # Save frame
            cv2.imwrite(os.path.join(dataset_dest_folder, 'features', csv.split('.')[0], frame_name), frame)
            
            success, image = vidcap.read()
            count += 1
        # Else we will adjsut the frames we process so we get the standard FPS rate
        else:
            # We will caluculate the second where we are on the video and scale it to the desired FPS
            out_due = int(frames_in / fps_in * fps)
    
            if out_due > frames_out:
                frames_out += 1
                # Build frame name
                frame_name = 'frame'
                for i in range(4-len(str(count))):
                    frame_name += '0'
                frame_name += str(count) + '.jpg'
        
                # Crop image so mouse is postioned in the center
                frame = crop_image(image, expansion)
                
                # Save frame
                cv2.imwrite(os.path.join(dataset_dest_folder, 'features', csv.split('.')[0], frame_name), frame) 
                count += 1  
            # We will remove those rows that don't correspond to any frame
            else:
                if count_df < len(df):
                    drop_indx.append(count_df)

            success, image = vidcap.read()
            frames_in += 1
            count_df += 1
            
    if  fps_in != fps:
        df = df.drop(drop_indx, axis=0)
        # Save df as csv
        df.to_csv(os.path.join(dataset_dest_folder, 'labels', csv.split('.')[0] + '.csv'), index=False)
        
    print("Label rows: ", df.shape)
    print("Video frames: ", count)

1.mp4
1.csv
DF:  (10884, 2)
Video FPS:  29.97
Label rows:  (3631, 2)
Video frames:  3631
2.mp4
2.csv
DF:  (12055, 2)
Video FPS:  29.97
Label rows:  (4022, 2)
Video frames:  4022
3.mp4
3.csv
DF:  (12795, 2)
Video FPS:  29.97
Label rows:  (4268, 2)
Video frames:  4269
4.mp4
4.csv
DF:  (13558, 2)
Video FPS:  29.97
Label rows:  (4523, 2)
Video frames:  4524
5.mp4
5.csv
DF:  (12178, 2)
Video FPS:  29.97
Label rows:  (4063, 2)
Video frames:  4063
Video_Animal61980_10min.mp4
Animal61980.csv
DF:  (6000, 2)
Video FPS:  10.0
Label rows:  (6000, 2)
Video frames:  6000
Video_Animal62418_10min.mp4
Animal62418.csv
DF:  (6000, 2)
Video FPS:  10.0
Label rows:  (6000, 2)
Video frames:  6000
